# Workforce schedeuling example

- Solve a classic model to assign workers to shifts
- This model is infeasible, so we do some tests to determine why

## List data
  1. Shifts
  2. Workers

In [1]:
from gurobipy import *

m = Model("workforce")

# Number of workers required for each shift
shifts = [ "Mon1",  "Tue2", "Wed3",  "Thu4",  "Fri5",  "Sat6",  "Sun7",
           "Mon8",  "Tue9", "Wed10", "Thu11", "Fri12", "Sat13", "Sun14" ]

workers = [ "Amy", "Bob", "Cathy", "Dan", "Ed", "Fred", "Gu" ]

Set parameter Username
Set parameter LicenseID to value 2588857
Academic license - for non-commercial use only - expires 2025-11-22


## Indexed data
  1. Shift requirements: indexed by shifts
  2. Worker pay: indexed by workers

In [4]:
shiftRequirements = {
"Mon1":  3, "Tue2":  2, "Wed3":  4, "Thu4":  4, "Fri5":  5, "Sat6":  6, "Sun7":  5,
"Mon8":  2, "Tue9":  2, "Wed10": 3, "Thu11": 4, "Fri12": 6, "Sat13": 7, "Sun14": 5 } 

# Amount each worker is paid to work one shift
pay = {
  "Amy":   10,
  "Bob":   12,
  "Cathy": 10,
  "Dan":   8,
  "Ed":    8,
  "Fred":  9,
  "Gu":    11 }

## Worker availability
We create a list of shifts when workers are available
- Much more efficient to list only shifts that are possible
- Don't optimize shifts for workers who are unavailable!

In [7]:
# Worker availability
availability = tuplelist([
('Amy', 'Tue2'), ('Amy', 'Wed3'), ('Amy', 'Fri5'), ('Amy', 'Sun7'),
('Amy', 'Tue9'), ('Amy', 'Wed10'), ('Amy', 'Thu11'), ('Amy', 'Fri12'),
('Amy', 'Sat13'), ('Amy', 'Sun14'), ('Bob', 'Mon1'), ('Bob', 'Tue2'),
('Bob', 'Fri5'), ('Bob', 'Sat6'), ('Bob', 'Mon8'), ('Bob', 'Thu11'),
('Bob', 'Sat13'), ('Cathy', 'Wed3'), ('Cathy', 'Thu4'), ('Cathy', 'Fri5'),
('Cathy', 'Sun7'), ('Cathy', 'Mon8'), ('Cathy', 'Tue9'), ('Cathy', 'Wed10'),
('Cathy', 'Thu11'), ('Cathy', 'Fri12'), ('Cathy', 'Sat13'),
('Cathy', 'Sun14'), ('Dan', 'Tue2'), ('Dan', 'Wed3'), ('Dan', 'Fri5'),
('Dan', 'Sat6'), ('Dan', 'Mon8'), ('Dan', 'Tue9'), ('Dan', 'Wed10'),
('Dan', 'Thu11'), ('Dan', 'Fri12'), ('Dan', 'Sat13'), ('Dan', 'Sun14'),
('Ed', 'Mon1'), ('Ed', 'Tue2'), ('Ed', 'Wed3'), ('Ed', 'Thu4'),
('Ed', 'Fri5'), ('Ed', 'Sun7'), ('Ed', 'Mon8'), ('Ed', 'Tue9'),
('Ed', 'Thu11'), ('Ed', 'Sat13'), ('Ed', 'Sun14'), ('Fred', 'Mon1'),
('Fred', 'Tue2'), ('Fred', 'Wed3'), ('Fred', 'Sat6'), ('Fred', 'Mon8'),
('Fred', 'Tue9'), ('Fred', 'Fri12'), ('Fred', 'Sat13'), ('Fred', 'Sun14'),
('Gu', 'Mon1'), ('Gu', 'Tue2'), ('Gu', 'Wed3'), ('Gu', 'Fri5'),
('Gu', 'Sat6'), ('Gu', 'Sun7'), ('Gu', 'Mon8'), ('Gu', 'Tue9'),
('Gu', 'Wed10'), ('Gu', 'Thu11'), ('Gu', 'Fri12'), ('Gu', 'Sat13'),
('Gu', 'Sun14')
])

## Decision variables
The variable `x[w,s] = 1` if worker w is assigned to shift s
- Only use variables for _possible_ combinations - not when workers are unavailable
- Reduces model size and makes code efficient

In [10]:
# Create variables indexed by availability list
x = m.addVars(availability, ub = 1, name = "x")

## Objective
Minimize total salary cost

In [13]:
m.setObjective(sum(pay[w]*x[w,s] for w,s in availability), GRB.MINIMIZE)

## Shift requirement constraints
For every shift, the number of workers must equal the shift requirement

In [16]:
reqCts = m.addConstrs((x.sum('*', s) == shiftRequirements[s] for s in shifts), '_')

To understand the syntax, here is a less efficient version:

    for s in shifts:
        m.addConstr(sum(x[w,s] for w in workers
                               if (w,s) in availability)
                    == shiftRequirements[s])

The accompanying presentation explains the efficiency issues

## Solve
Remember that this model is designed to be infeasible

In [20]:
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14 rows, 72 columns and 72 nonzeros
Model fingerprint: 0xc4a4a254
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 7e+00]
Presolve removed 1 rows and 7 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Infeasible model


## Relax constraints
Call feasRelax to relax constraints

In [23]:
orignumvars = m.NumVars
m.feasRelaxS(0, False, False, True)
m.optimize()

m.printAttr('X', "Art*") # slack variables

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14 rows, 100 columns and 100 nonzeros
Model fingerprint: 0x8bac3844
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 7e+00]
Presolve removed 5 rows and 73 columns
Presolve time: 0.01s
Presolved: 9 rows, 27 columns, 27 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.0000000e+00   0.000000e+00   0.000000e+00      0s
       0    6.0000000e+00 

## Plot result
Show a heatmap of the shifts that cannot be satisfied

In [ ]:
from bokeh.charts import *

output_notebook()

slackvars = m.getVars()[orignumvars:]
data = {
    'days': [s[:3] for s in shifts],
    'weeks': [1+i/7 for i in range(len(shifts))],
    'slacks': [v.X for v in slackvars if v.VarName.startswith("ArtP")],
}

show(HeatMap(data, x='days', y='weeks', values='slacks', stat=None, title="Shortfall"))

In [28]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, ColorBar
from bokeh.transform import linear_cmap
from bokeh.palettes import Viridis256
import pandas as pd

output_notebook()

# Originaldaten vorbereiten
slackvars = m.getVars()[orignumvars:]
data = {
    'days': [s[:3] for s in shifts],
    'weeks': [1 + i / 7 for i in range(len(shifts))],
    'slacks': [v.X for v in slackvars if v.VarName.startswith("ArtP")],
}

# In DataFrame umwandeln für Bokeh-Source
df = pd.DataFrame(data)
source = ColumnDataSource(df)

# Farbcodierung für die Slacks
mapper = linear_cmap(field_name='slacks', palette=Viridis256, low=df.slacks.min(), high=df.slacks.max())

# Plot vorbereiten
p = figure(title="Shortfall", x_range=sorted(df['days'].unique()), y_axis_label="Week", tools="hover", tooltips=[("Slack", "@slacks")], width=600, height=400)
p.rect(x="days", y="weeks", width=1, height=0.15, source=source, fill_color=mapper, line_color=None)

# Farbskala hinzufügen
color_bar = ColorBar(color_mapper=mapper['transform'], width=8, location=(0,0))
p.add_layout(color_bar, 'right')

show(p)


Loading BokehJS ...